In [1]:
# %matplotlib
%pylab
import numpy as np

import sys
sys.path.append(r'D:\opencv249\sources\modules\python\test')

import os
import cv2
import transformations as tf
'一些短别名:'
tf.e2m = tf.euler_matrix
tf.m2e = tf.euler_from_matrix
tf.q2m = tf.quaternion_matrix
tf.m2q = tf.quaternion_from_matrix
tf.e2q = tf.quaternion_from_euler
tf.q2e = tf.euler_from_quaternion
tf.m2r = tf.rotation_from_matrix
tf.r2m = tf.rotation_matrix
tf.a2q = tf.quaternion_about_axis

'angle <--> radian:'
a2r = np.pi / 180
r2a = 180 / np.pi


Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [7]:
'''
对应数据采集程序：
https://github.com/zhangxaochen/AndroidSensorFusion.git
E:\Github\AndroidSensorFusion

目的：
1. 观察三轴单轴旋转，输出的rot-vec姿态正确性；
2.
'''

'旧数据: 16列, deprecated'
'手机【+X】轴朝北, 依次绕xyz逆(+)顺(-)旋转90°'
fn = r'D:\Users\zhangxaochen\Desktop\androidSFzc-20151214T142422.txt'

'手机【-Y】轴朝北, 依次绕xyz逆(+)顺(-)旋转90°'
# fn = r'D:\Users\zhangxaochen\Desktop\androidSFzc-20151214T142511.txt'

dat = np.loadtxt(fn, delimiter=',')

goog = dat[:, 1:4]
myOri = dat[:, 10:13]
plot(goog, label='android') #android rotation-vector
# plot(dat[:, 4:7]) #accMagOrientation
# plot(dat[:, 7:10]) #gyroOrientation
plot(myOri, label='myOri') #fusedOrientation； 结论：与 gyroOrientation 基本重合
legend()


In [20]:
'格式改为22列, 13~21为a/g/fQuaternion'
'手机+X朝北，没有移动'
fn = r'D:\Users\zhangxaochen\Desktop\androidSFzc-20151215T221552.txt'
'同上，7000frames'
fn = r'D:\Users\zhangxaochen\Desktop\androidSFzc-20151216T003244.txt'
'+X, north; 依次绕xyz逆(+)顺(-)旋转90°'
fn = r'D:\Users\zhangxaochen\Desktop\androidSFzc-20151216T005812.txt'
'+Y, north; 依次绕xyz逆(+)顺(-)旋转90°'
fn = r'D:\Users\zhangxaochen\Desktop\androidSFzc-20151216T005837.txt'

'格式改为25列, 13~24为android-rot-vec && a/g/fQuaternion'
'''
重新:: +Y, north; 依次绕xyz逆(+)顺(-)旋转90°
此数据结论： 1. Android-rotVec 更平滑; 2. Android && fQua 的可视化结果(axag)都基本正确
'''
fn = r'D:\Users\zhangxaochen\Desktop\androidSFzc-20151216T014041.txt'
'+X, north, 旋转同上。结果：效果不好，可能因为插了USB干扰'
# fn = r'D:\Users\zhangxaochen\Desktop\androidSFzc-20151216T150358.txt'
'''
拔掉USB重采，+X, north, 旋转同上。
结论: 发现 dri表达式推错了, 改为 ΔRi = r0^-1*ri, √
'''
fn = r'D:\Users\zhangxaochen\Desktop\androidSFzc-20151216T152354.txt'

dat = np.loadtxt(fn, delimiter=',')

'delta euler angles i/0'
deuls = []
'delta AxisAngle i/0'
daxags = []

'fused quaternion'
# fQuas = dat[:, 19:22] #22列时候, fQua
fQuas = dat[:, 22:25] #25列时候, fQua
fQuas = dat[:, 13:16] #25列， 13为Android-Qua
fq0 = fQuas[0]
w0 = sqrt(1-norm(fq0))
r0 = tf.q2m(append(fq0, w0))

for i, fqi in enumerate(fQuas):
    wi = sqrt(1-norm(fqi))
    '第i帧rotMat'
    ri = tf.q2m(append(fqi, wi))
    'ΔRi = ri*r0^-1 //×'
    dri=ri.dot(r0.T) #可能弄错，不该是 Ri*R0^-1, 而是 Ri^-1*R0
    'ΔRi = ri^-1*r0 //√×'
    dri=ri.T.dot(r0) #还不太对, 这是第i帧的“变换”而非“姿态”矩阵： vib = Ri^-1*R0*v0b
    'ΔRi = r0^-1*ri //√, vib->v0b, 是第i帧姿态矩阵'
    dri=r0.T.dot(ri)
#     dri=ri #若用原始 ri呢？

    'AndroidSensorFusion 程序中用 zxy顺序存储ori、合成rotMat'
    ez, ex, ey = tf.m2e(dri, axes='rzxy')
    deuls.append([ex, ey, ez])
#     deul = tf.m2e(dri, axes='rxyz') #与上面肯定不一样， 只是 20151216T003244.txt 移动太小了，虽然确实太相似
#     deuls.append(deul)
    
    ag, ax, _ = tf.m2r(dri)
    daxags.append(np.append(ax, ag))
    pass

deuls = np.array(deuls)
daxags = np.array(daxags)

figure()
subplot(311)
plot(deuls)
legend(list('xyz'))
subplot(312)
plot(daxags)
legend(list('xyz')+['ag'])

'''
---------------★ 顺便测试 fOri 用 py 转为轴角, 是否与Android的fQua->axag一致
结论: 1. daxags2 不等 daxags, 因为Android输出的fOri坐标系定义非ENU, 而是WND
'''
'轴角序列, 对照组'
daxags2 = []

fOris = dat[:, 10:13] #22or25列， 10为fOri eulerAngle
fo0 = fOris[0]
'按照Android的 zxy-orientation顺序, ->Qxyzw'
axes='rzxy'
fq0 = tf.e2q(*fo0, axes=axes)
r0 = tf.q2m(fq0)

for i, foi in enumerate(fOris):
    fqi = tf.e2q(*foi, axes=axes) #Qxyzw
    ri = tf.q2m(fqi)
    dri = ri.dot(r0.T)
    
    ag, ax, _ = tf.m2r(dri)
    daxags2.append(np.append(ax, ag))
    pass
daxags2 = np.array(daxags2)

subplot(313)
plot(daxags2)
legend(list('xyz')+['ag'])

print 'np.allclose(daxags, daxags2): ', np.allclose(daxags, daxags2) #False

np.allclose(daxags, daxags2):  False


In [18]:
'''
使用@王镇 手机采集的 相机+IMU数据做AX=XB算法测试
1. 预处理，整理为之前axxb.m可接受文件格式

'''

import os, sys
from datetime import datetime
from lxml import etree as et
from StringIO import StringIO

'@王镇 数据工程目录'
projPath = r'D:\Users\zhangxaochen\Documents\axxb-data\Domke\NVan'
projPath = r'D:\Users\zhangxaochen\Documents\axxb-data\Domke\Lobo81'
projPath = r'D:\Users\zhangxaochen\Documents\axxb-data\Domke\Lobo82'
projPath = r'D:\Users\zhangxaochen\Documents\axxb-data\Domke\Lobo83'
projPath = r'D:\Users\zhangxaochen\Documents\axxb-data\Domke\Lobo84'
projPath = r'D:\Users\zhangxaochen\Documents\axxb-data\Domke\Lobo88'

'proj-description-xml'
projXml = os.path.join(projPath, 'collection-proj.xml')

psr = et.XMLParser(remove_blank_text = True)
tree = et.parse(projXml, psr)
rt = tree.getroot()

cnameNode = rt[0]
collsNode = rt[1]
picCnt = len(collsNode)
print '一组几张图？ picCnt:', picCnt

'存放 imu找到的对应pic时间戳的rotVec-rodrigues表示'
imuRodrs = np.zeros((picCnt, 3))

for i, cNode in enumerate(collsNode):
#     if i>0: #调试用，注释掉！
#         break
    #print cNode
    sensorName = cNode[0].text
    picsNode = cNode[1]
    '只取第0个, 此数据也只有1个(idx=0)'
    picNode = picsNode[0]
    picName = picNode[0].text
    picTs = float(picNode[1].text)
    #print sensorName, picName, picTs, datetime.fromtimestamp(picTs)
    sensorFpath = os.path.join(projPath, sensorName)
    #print sensorFpath
    assert os.path.exists(sensorFpath) #理论上每个对应文件都应该存在
    
    '以下解析 sensor_xxx.xml, 此xml组织结构稍复杂'
    s_tree = et.parse(sensorFpath)
    s_rt = s_tree.getroot()
    'channels-node'
    cnsNode = s_rt.xpath(u'threads/thread/channels')[0]
    
    '直接用android-rotVec, '
    cnRotNode = cnsNode[3]
    #print 'cnRotNode: %s: %s'%(cnRotNode[0].tag, cnRotNode[0].text)
    #print cnRotNode.xpath(u'frames/frame') #复杂，不必要
    #print 'frame-count:', cnRotNode[2].text
    framesNode = cnRotNode[1]
    for j, frmNode in enumerate(framesNode):
        frmTs = float(frmNode[2].text)
        #print 'frmTs: %f, %s'%(frmTs, datetime.fromtimestamp(frmTs))
        if frmTs > picTs:
            assert j > 0, 'j must NOT be of index 0, maybe data is INVALID!!'
            prevFrmNode = framesNode[j-1]
            prevFrmTs = float(prevFrmNode[2].text)
            assert prevFrmTs < picTs, 'logically, should be: (prevFrmTs < picTs); %f, %f'%(prevFrmTs, picTs)
            'value-node 为 quaternion, wxyz order'
            valNode = prevFrmNode[0]
            q = [e.text for e in valNode] #wxyz order
            q = q[1:]+q[:1] #->xyzw order
            rodr, _ = cv2.Rodrigues(tf.q2m(q)[:3,:3])
            '存起来'
            imuRodrs[i] = rodr.ravel()
            break
        pass
    pass


'---------------解析cam.xml, 获取cv定标板得到的外参'
cvRodrs = np.zeros_like(imuRodrs)
cvExtrFname = os.path.join(projPath, 'cam.xml')
assert os.path.exists(cvExtrFname)

psr = et.XMLParser(remove_blank_text = True, remove_comments=True)
tree = et.parse(cvExtrFname, psr)
rt = tree.getroot()
cameraMatrixNode = rt[8]
# print cameraMatrixNode
distortionCoefficientsNode = rt[9]
cvExtrParamsNode = rt[12]
cvExtrParamsDataNode = cvExtrParamsNode[3]
# print extrParams.text
cvExtrParams = np.genfromtxt(StringIO(cvExtrParamsDataNode.text)) #shape=(N,2)
cvExtrParams = cvExtrParams.reshape(-1, 6)

'---------------二者合并, cv(R,t)+imu(R,t), 12列'
csvMat = np.hstack([cvExtrParams, imuRodrs, np.zeros_like(imuRodrs)])
cvAndImuExtrPath = os.path.join(projPath, 'cvAndImuExtr.csv')
np.savetxt(cvAndImuExtrPath, csvMat, fmt='%f')

一组几张图？ picCnt: 15


In [19]:
'''
只是预处理csv文件, 将@王镇生成的姿态（rodrigues）塞到对应csv中 //2015-12-22 15:03:11
'''

'对应 NVan数据'
wzAxags=np.array([
[0.5334803200676891, -0.3865801906299841, 0.1087840576315074],
[0.7053919183743225, -0.1785478302412343, -0.185786395791522],
[0.8076928515254848, -0.2262407179579445, -0.1831060524219923],
[0.9031431161418061, -0.2641238636360845, -0.1785235871622655],
[0.3830242241069002, -0.4734496162970422, 1.100092789510663],
[0.4762464762194099, -0.6142321840524212, 1.099555373639786],
[0.1137065575511585, -0.7058807157464764, 0.1287876089072303],
[0.116266191201658, -0.8878740962410876, 0.1287751318075772],
[-0.03075896543205586, -0.7357280665787732, -0.2422862090322829],
[-0.03723299496021638, -0.7929588352737982, -0.2352213713459948],
[0.4319138677623127, -0.8223045617912043, -1.785272552792782],
[0.07900589101743498, -1.064101317043946, -2.545267202034877],
[-0.2288433674647769, -0.8451185645855874, -2.077019360186496],
[-0.3829599051948317, -0.6005064891454988, -1.395842740401545],
[-0.8912357342490772, -0.3864155767268521, -2.340300489239013],
[-0.6802985481441392, -0.3195816437555551, -2.415341794389918],
[0.3287977150851776, 0.8839901535502893, 2.480087458603597],
[0.5229394080430097, 1.227855465692769, 2.448170984628599],
[0.2173645267680565, 0.7166118660615748, 2.672847713266102],
[0.1209152167438263, 0.6953457553227604, 2.432223713830392],
[0.4755485766162302, 0.2274927566005761, 0.5607144954266473],
[0.7188010934255943, 0.2929494815265206, 0.5778444248162084],
[0.8822118594441202, 0.3492315933833771, 0.5889352989203464],
[0.9587989698346828, -0.2819053040288969, 2.247706216193234],
[0.7426486350022299, -0.1890852826847766, 2.300725851721483],
[0.6413702793124961, -0.1450020374376749, 2.321979437562247],
[0.7128265417544504, 0.2186210185007338, 1.329635949363218],
[0.8246091819190583, 0.1726329538249485, 1.2191838002252],
        ])

'原始第一次生成的12列, NVan数据, opencv-calib结果 + androidIMU直接输出姿态'
origABcsvFn=r'D:\Users\zhangxaochen\Documents\axxb-data\Domke\NVan\cvAndImuExtr.csv'

wzAxags=np.array([
[1.077160111246911, -1.176296449480052, -1.832354085124013],
[0.9298751990929581, -1.379190688369149, -1.691245906577581],
[0.7836064228034191, -1.551955082086034, -1.475169290233496],
[0.6646818497148923, -1.643950976751684, -1.22612379019221],
[0.5380876860613243, -1.644029775199617, -0.9300385239028828],
[0.3112745671229941, -1.671851328835502, -0.6068589262916281],
[0.04493255830606015, -1.65655464907741, -0.2373191135115994],
[-0.4960887369387219, -1.513620144962189, 0.02885446171394058],
[-0.713740213341626, -1.481257297772469, 0.2594895990278507],
[-1.04705252483882, -1.394885723309144, 0.6163426608679866],
[-1.380476585750755, -1.240574953443424, 0.9491950234650678],
[-0.2349290953399285, -1.737278700363026, -0.322930923062992],
[-0.2518804873219411, -1.606606448669058, -0.2805521204097293],
[-0.2839211288395007, -1.466070339393462, -0.2503618421926679],
[-0.2885971233201577, -1.407449261391483, -0.2249575007904265],
    ])
origABcsvFn=r'D:\Users\zhangxaochen\Documents\axxb-data\Domke\Lobo81\cvAndImuExtr.csv'

wzAxags=np.array([
[-0.8305420189586079, -1.498728352230724, -0.7829306800864784],
[-0.8074406405237942, -1.438878596732705, -0.7914191478180878],
[-0.76545890275003, -1.313357611728387, -0.8114745906487481],
[-0.9660518309700809, -1.302641564926533, -0.9844004246542017],
[0.580058533050517, -1.987993474898003, -1.805353314609604],
[0.5080850585136252, -1.68400748371013, -2.030224398189233],
[0.2074438049605314, -1.815147035657822, -1.851738169509698],
[-0.1948319367490056, -1.747822376004263, -1.695729882772728],
[-0.5436931590548239, -1.607704478778566, -1.485581223436972],
[-0.9605202240124631, -1.328351655925519, -1.097906673748492],
[-1.244268460389916, -1.047506278856531, -0.7153141849176086],
[-1.433439773799599, -0.793074658986446, -0.4532076349487199],
[-1.609657739362592, -0.4157524784353863, -0.1458452096531342],
[-1.688407314442242, 0.1338085969929001, 0.1926548039851933],
[-1.685380222614469, 0.2209263026557591, 0.2300184772341999],
    ])
origABcsvFn=r'D:\Users\zhangxaochen\Documents\axxb-data\Domke\Lobo82\cvAndImuExtr.csv'

wzAxags=np.array([
[-1.591063009196307, 0.4960482656444885, -0.03598318742281975],
[-1.638813243518487, 0.2494142674563385, -0.1923861268003577],
[-1.646067244177747, -0.1072541963999226, -0.4248370478282567],
[-1.561202510527099, -0.5253070235006757, -0.7214421380094004],
[-1.368289901766348, -0.9570867551256822, -1.136720860095836],
[-1.033499779460835, -1.400099180601076, -1.545334943678538],
[-0.6895077185423746, -1.689966745777676, -1.798298045428792],
[-0.3614975229588323, -1.868416339206818, -1.950224635840662],
[-1.395227962459452, -1.111006839585548, -1.402817817243359],
[-1.313446772457585, -0.9608726059522811, -1.447178086381502],
[-1.421973205273163, -1.152242544273419, -1.395136534756948],
[-1.315730497797949, -1.166874013479898, -1.325172307243864],
[-1.157214322690928, -1.267603270552862, -1.174199039069801],
    ])
origABcsvFn=r'D:\Users\zhangxaochen\Documents\axxb-data\Domke\Lobo83\cvAndImuExtr.csv'

wzAxags=np.array([
[-1.736039173779268, -0.423132367105162, -2.040123259475304],
[-1.788886904530965, -0.5524079552448259, -1.933963695573377],
[-1.837231793881775, -0.7127393625952672, -1.803701542420695],
[-1.888460750136373, -0.8095150543134603, -1.728819832208552],
[-1.635951672737411, 0.5472794786610058, -0.5240866256969945],
[-1.714483311559814, 0.3085531563288075, -0.721203169357156],
[-1.753709066369341, 0.1199369817115971, -0.8815354497260679],
[-1.788398342449352, -0.1343403159010361, -1.167179252339833],
[-1.804632491242723, -0.2973494175305756, -1.368674511944054],
[-1.829060957099716, -0.5801567461287746, -1.64141228822991],
[-1.791603754307635, -0.8654850998773801, -1.883137609085275],
[-1.719467301296252, -1.15713828835407, -2.09517405462502],
[-1.638480917897718, -1.378329761254709, -2.2215850306968],
[1.279165340462302, 1.605885929277973, 2.179444962537888],
[0.6701453732012297, 1.757346619250092, 1.960271638052505],
    ])
origABcsvFn=r'D:\Users\zhangxaochen\Documents\axxb-data\Domke\Lobo84\cvAndImuExtr.csv'

wzAxags=np.array([
[1.11050702434583, 1.417780334336144, 1.337492912717788],
[1.009761243247666, 1.530347842819711, 1.463806282291029],
[1.241479971051679, 1.377797521414348, 1.567946997493966],
[1.497410036129813, 1.201881198646488, 1.76438057804979],
[1.796486214754058, 0.9221226810564478, 1.990441709140296],
[2.137057161164248, 0.4522360972807205, 2.236523904049933],
[-1.959641544110549, 0.3665680152657944, -1.969202171215522],
[-1.611844709167281, 0.8962587518380505, -1.620626722799962],
[-1.256180049076926, 1.181936718215921, -1.177062098316325],
[-1.504284714166021, 0.9432542570960854, -1.411006715853704],
[2.064810664966269, -0.199527465397965, 2.213256767214927],
[1.902225945141499, -0.2155681112683539, 2.316343050380323],
[2.044787838242465, -0.09083471034396506, 2.347757859381001],
[2.148738457119019, -0.06027357772279465, 2.284865030484447],
[1.969509748503262, -0.08866324021860665, 2.415159612091197],
    ])
origABcsvFn=r'D:\Users\zhangxaochen\Documents\axxb-data\Domke\Lobo85\cvAndImuExtr.csv'

wzAxags=np.array([
[1.035864442121814, -1.09326355188943, 1.280869102988985],
[1.116345230236652, -1.152035312161637, 1.257213820550504],
[1.192576398710302, -1.213505865270553, 1.22366448500634],
[1.22050823453762, -1.264104759000932, 1.182301159944457],
[1.464847793174819, -1.076709553282006, 1.426306090535549],
[1.323585442460291, -0.9896255880355942, 1.505449146626164],
[1.237004115987652, -0.9470675106002081, 1.531860626722935],
[1.512367333686958, 0.1007381593559196, 0.1373628223349727],
[1.521365096618659, -0.03125120533193977, 0.5375907763612322],
[1.540405515300834, -0.2411572523901441, 0.8624103863674256],
[1.452652505737031, -0.7958148921650979, 1.258316108245991],
[1.227482940179121, -1.225975635836795, 1.43623493891081],
[0.8662506019966969, -1.624877162079551, 1.618367067913355],
[0.3789898406259786, -2.000336949704565, 1.885271899173095],
[0.06616939648359035, 2.220114664968028, -2.052078778193759],
    ])
origABcsvFn=r'D:\Users\zhangxaochen\Documents\axxb-data\Domke\Lobo86\cvAndImuExtr.csv'

wzAxags=np.array([
[1.367468864580607, -0.5761819147907263, -0.634116491113879],
[1.517207746340018, -0.6568445641987483, -0.5951160100954124],
[1.389623919912942, -0.7356648705676266, -0.3284998663086003],
[1.235351174263997, -0.9307639316136158, 0.08389684035094791],
[1.073411464803362, -1.115222720451142, 0.3172511511388917],
[0.8423110805217237, -1.31086874657409, 0.5342892169747087],
[0.596054493351238, -1.459237087927814, 0.7106548461868161],
[0.2478541603942648, -1.604098622518989, 0.9698506337650137],
[0.0235716069373027, -1.667278900154732, 1.132479016962292],
[-0.2624419318082007, -1.734149055409546, 1.396983950294678],
[-0.618408220249866, -1.78637628096526, 1.704092460587727],
[0.6482593841403977, -1.384784184817947, 0.7238727003912701],
[0.7099081112296685, -1.457667107053861, 0.6663870676358593],
[0.7578559759804684, -1.500515537156355, 0.6400226987863334],
[1.00134988625432, -1.38801722631253, 0.8757458047186522],
[0.8643136148865254, -1.292747611058393, 0.9737607290489568],
    ])
origABcsvFn=r'D:\Users\zhangxaochen\Documents\axxb-data\Domke\Lobo87\cvAndImuExtr.csv'

wzAxags=np.array([
[0.2228127694447382, -1.45108932107671, 0.284913389012054],
[0.2870786484450356, -1.522086282127581, 0.2826239091472163],
[0.3299567896883086, -1.576240486038494, 0.2688946050886792],
[0.3556354736068141, -1.613926238339661, 0.2542780875385046],
[1.343175926595936, -0.780629938164926, -1.307402754270169],
[1.215938224575079, -0.8992819771592033, -1.136755238337601],
[1.008914785065489, -1.067956711065727, -0.8934337244429199],
[0.701412918411339, -1.273552170191401, -0.4914343113645235],
[0.5497263188834243, -1.415557041733626, 0.363555854482105],
[0.3468450073635045, -1.575489796804942, 0.6758771181366973],
[0.2326771592471351, -1.672609434965821, 0.8784246577150524],
[-0.3477469208346412, -2.13770689549665, 1.728304140040126],
[-0.3220765927238345, -1.762164366600091, 2.018701418379901],
[0.3620276571902646, -1.565600152938339, 0.6287106571005771],
[0.6906384655671177, -1.360132481253104, 0.09321300008535174],
    ])
origABcsvFn=r'D:\Users\zhangxaochen\Documents\axxb-data\Domke\Lobo88\cvAndImuExtr.csv'


'---------------替换原来android-IMU姿态，输出到文件'
origABcsv=np.loadtxt(origABcsvFn, delimiter=' ')

newABmat=origABcsv.copy()
newABmat[:, 6:9]=wzAxags

baseFpath, ext=os.path.splitext(origABcsvFn)
newFn=baseFpath+'.wz'+ext
np.savetxt(newFn, newABmat, fmt='%f')

In [179]:
wzAxags.shape

(28, 3)

In [161]:
print csvMat.shape
print cvExtrParams.shape, imuRodrs.shape

(28, 12)
(28, 6) (28, 3)


In [176]:
print arccos(0.999361218)*2*r2a
print arccos(0.999808157)*2*r2a
print arccos(0.986935574)*2*r2a


4.09606728262
2.24464439273
18.5432881263


In [42]:
[nd[0].text for nd in s_rt.xpath(u'threads/thread/channels/channel')]

['a', 'g', 'm', 'r']

In [11]:
collsNode.xpath

<function xpath>